Run the below cell to download the MuJoCo dependencies

In [ ]:
!pip install -U gymnasium[mujoco] mujoco

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# **IMPORTING LIBRARIES**

In [ ]:
import torch
import random
import os
import torch.nn as nn
import numpy as np
import time
from collections import deque
import wandb

import gymnasium as gym
# from gymnasium.vector import SyncVectorEnv
from gymnasium.wrappers import RecordVideo

In [ ]:
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
# def create_environment(cfgs, eval = False):

#   def _init():
#       env = gym.make( id=cfgs.id , render_mode="rgb_array", max_episode_steps=cfg.max_steps)
#       return env

#   return _init

In [ ]:
def create_environment(cfgs, eval = False):
  env = gym.make( id=cfgs.id , render_mode="rgb_array", max_episode_steps=cfg.max_steps)
  return env

# **WANDB RUN**

In [ ]:
def wandb_runs(cfg):

  wandb.login(key = "")
  run = wandb.init(
    entity="ajheshbasnet-kpriet",
    project="ddpg",
    name = "DDPG",
    config=vars(cfg),
  )

  return run

# **CONFIGURATIONS**

In [ ]:
from dataclasses import dataclass

@dataclass
class configuration:
  id = "HalfCheetah-v5"
  n_rollouts = 100_000
  max_steps = 1000
  eval_steps = 10_000
  global_steps = 0
  buffer_size = 800_000
  eval_loops = 3
  batch_size = 512
  wandb_log_steps = 50
  start_training = 80_000
  trainng_step = 1
  critic_lr = 5e-4
  actor_lr = 5e-4
  record_video = 500_000
  eval_max_steps = 800
  device = "cuda" if torch.cuda.is_available() else "cpu"

cfg = configuration()

**SyncVectorEnv so that we can run the n-environments parrallelly and utilize the GPUs because single environment is wayy poor**

In [ ]:
# envs = SyncVectorEnv([create_environment(cfg) for _ in range(cfg.n_envs)])

envs = create_environment(cfg)

**Checking environment is working or not:)**

In [ ]:
envs.reset()[0]

# **Actor and Critic Netowrk**

In [ ]:
class Actor(nn.Module):

  def __init__(self, input_dim, action_dim):
    super().__init__()
    self.sequential = nn.Sequential(
        nn.Linear(input_dim, 400),
        nn.ReLU(),
        nn.Linear(400, 300),
        nn.ReLU(),
        nn.Linear(300, action_dim),
        nn.Tanh()
    )

  def forward(self, x):
    x = self.sequential(x)
    return x

In [ ]:
class Critic(nn.Module):

  def __init__(self, input_dim):
    super().__init__()

    self.sequential = nn.Sequential(
        nn.Linear(input_dim, 400),
        nn.ReLU(),
        nn.Linear(400, 300),
        nn.ReLU(),
        nn.Linear(300, 1)
    )

  def forward(self, state, action):
    x = torch.cat([state, action], dim = 1)
    x = self.sequential(x)
    return x

In [ ]:
print(envs.observation_space,"\t", envs.action_space,)

In [ ]:
actornet = Actor(envs.observation_space.shape[0], envs.action_space.shape[0]).to(cfg.device)  #type: ignore
criticnet = Critic(envs.observation_space.shape[0]+envs.action_space.shape[0]).to(cfg.device)  #type: ignore

TargetActor = Actor(envs.observation_space.shape[0], envs.action_space.shape[0]).to(cfg.device) #type: ignore
TargetCritic = Critic(envs.observation_space.shape[0]+envs.action_space.shape[0]).to(cfg.device) #type: ignore

TargetActor.load_state_dict(actornet.state_dict())
TargetCritic.load_state_dict(criticnet.state_dict())

In [ ]:
print(f'''Parameters:
===========================
actor-network :  {sum(p.numel() for p in actornet.parameters())/1e3} k
critic-network : {sum(p.numel() for p in criticnet.parameters())/ 1e3} k
===========================
      ''')

**Evaluation Loop**

In [ ]:
def evaluation(actornet, record_video = False):

  eval_env = gym.make(id = cfg.id, render_mode = 'rgb_array' ,max_episode_steps=cfg.eval_max_steps)
  if record_video:
    video_dir = f"videos/{int(time.time())}"
    eval_env = RecordVideo(eval_env,  video_folder=video_dir, episode_trigger=lambda ep: True)

  net_reward = 0
  net_step = 0

  with torch.no_grad():

    for _ in range(cfg.eval_loops):

      done = False

      episodic_reward = 0
      episodic_step = 0
      state = eval_env.reset()[0]

      while not done:

        stateT = torch.tensor(state, dtype=torch.float32, device=cfg.device)
        action = np.array(actornet(stateT).cpu().numpy())
        nxt_state, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated
        state = nxt_state

        episodic_reward += float(reward)
        episodic_step += 1

      net_reward += episodic_reward
      net_step  += episodic_step

  net_reward = net_reward / cfg.eval_loops
  net_step = net_step / cfg.eval_loops

  eval_env.close()

  return net_reward, net_step

In [ ]:
evaluation(actornet, False)

**To sample the batches**

In [ ]:
def get_batches(memory, batch_size):
    batches = random.sample(memory, batch_size)
    state, action, reward, next_state, done = zip(*batches)

    state = torch.stack(state).float().to(cfg.device)
    action = torch.stack(action).float().to(cfg.device)
    reward = torch.stack(reward).float().to(cfg.device)
    next_state = torch.stack(next_state).float().to(cfg.device)
    done = torch.stack(done).float().to(cfg.device)  # float for TD computation

    return state, action, reward.view(-1, 1), next_state, done.view(-1, 1)

# **REPLAY MEMORY**

In [ ]:
replay_buffer = deque(maxlen = cfg.buffer_size)
action_sigma = 0.1
tau = 0.001
gamma = 0.99
global_step = cfg.global_steps

In [ ]:
critic_optimizer = torch.optim.AdamW(criticnet.parameters(), lr = cfg.critic_lr)
actor_optimizer = torch.optim.AdamW(actornet.parameters(), lr = cfg.actor_lr)

**W&B RUNS TO LOG THE METRICS**

In [ ]:
runs = wandb_runs(cfg)

# **Heart & Core of the notebook: DDPG Algorithm's Training Loop**

In [ ]:
from tqdm import tqdm

for _ in tqdm(range(cfg.n_rollouts)):

  states = envs.reset()[0]

  statesT= torch.tensor(states, dtype=torch.float32, device = cfg.device)

  training_rewards = 0

  for _ in range(cfg.max_steps):

    with torch.no_grad():
      action = actornet(statesT).view(-1)

    action = (action + action_sigma * torch.randn(6).to(cfg.device)).cpu().numpy()

    next_states, rewards, terminated, truncated, _ =  envs.step(action)

    done = terminated | truncated

    next_statesT = torch.tensor(next_states, dtype=torch.float32, device = cfg.device)

    actionT = torch.tensor(action, dtype=torch.float32, device = cfg.device)

    rewardsT = torch.tensor(rewards, dtype=torch.float32, device=cfg.device)

    training_rewards += float(rewards)

    doneT = torch.tensor(done, dtype=torch.bool, device = cfg.device)

    replay_buffer.append((statesT, actionT, rewardsT, next_statesT, doneT))

    if (global_step + 1) % cfg.trainng_step == 0 and len(replay_buffer)>cfg.start_training:
      # Sample batch
      states_b, action_b, reward_b, next_states_b, dones_b = get_batches(replay_buffer, cfg.batch_size)

      # Target Q
      with torch.no_grad():
        next_action = TargetActor(next_states_b)
        target_next_q = TargetCritic(next_states_b, next_action)
        target_q = reward_b + gamma * target_next_q * (1 - dones_b.float())

      # Current critic Q
      current_q = criticnet(states_b, action_b)

      # Critic loss
      critic_loss = torch.nn.functional.mse_loss(current_q, target_q)

      # Optimize critic
      critic_optimizer.zero_grad()
      critic_loss.backward()
      critic_grad_norm = torch.nn.utils.clip_grad_norm_(criticnet.parameters(), max_norm=1.0)
      critic_optimizer.step()

      # Actor loss (use current actor)
      # Clone states_b to create an independent computational graph for actor update
      states_b_actor = states_b.clone()
      actor_actions = actornet(states_b_actor) # Renamed to avoid shadowing action_b from get_batches
      actor_loss = -criticnet(states_b_actor, actor_actions).mean()

      for p in criticnet.parameters():
        p.requires_grad = False

      # Optimize actor
      actor_optimizer.zero_grad()
      actor_loss.backward()
      critic_grad_norm = torch.nn.utils.clip_grad_norm_(criticnet.parameters(), max_norm=1.0)
      actor_optimizer.step()

      for p in criticnet.parameters():
        p.requires_grad = True


      advantages = (target_q - current_q).detach().mean()

      runs.log(
          {
              "actor-loss": actor_loss.item(),
              "critic-loss": critic_loss.item(),
              "advantages": advantages.item()
          }
      )

      # Soft update targets
      for target_param, param in zip(TargetCritic.parameters(), criticnet.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

      for target_param, param in zip(TargetActor.parameters(), actornet.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

      if global_step%cfg.eval_steps==0 and global_step>0:

        rec = True if global_step%cfg.record_video==0 else False
        eval_reward, eval_steps = evaluation(actornet, rec)
        runs.log(
            {
                "eval-reward": eval_reward,
            }
        )

    runs.log({"training-reward": training_rewards, "global-steps": global_step, "memory": len(replay_buffer)})

    statesT = next_statesT
    global_step += 1

envs.close()
wandb.finish()

**Run the below cell to save the video from the latest Actor Network**

In [ ]:
evaluation(actornet, True)

# **                         **END****